In [12]:
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

In [13]:
DATA_PATH = r'.\dataset\dota_dataset'
OUTPUT_PATH = r'.\dataset\dota_split_dataset'

In [90]:
def split_image(filename):
    print(filename)
    image_file = os.path.join(DATA_PATH, 'images', filename + '.png')
    label_file = os.path.join(DATA_PATH, 'labels', filename + '.txt')
    image = cv2.imread(image_file)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label_list = []
    with tf.gfile.GFile(label_file, 'r') as f:
        labels = f.readlines()
        for label in labels:
            if label.find('plane') != -1 or label.find('helicopter') != -1:
                label = label.strip().split(r' ')
                label_list.append({
                    'startX':min(int(float(label[0])),int(float(label[4]))),
                    'startY':min(int(float(label[1])),int(float(label[5]))),
                    'endX':max(int(float(label[0])),int(float(label[4]))),
                    'endY':max(int(float(label[1])),int(float(label[5]))),
                    'type':label[8]
                })
    xList = [ x for x in range(0, image.shape[1] - 500, 250)]
    yList = [ y for y in range(0, image.shape[0] - 500, 250)]
    xList.append(image.shape[1] - 500)
    yList.append(image.shape[0] - 500)
    count = 0
    canvas = image.copy()
    for x in xList:
        for y in yList:
            exist_flag = False
            image_output_path = os.path.join(OUTPUT_PATH, 'images', filename + '-' + str(count) + '.jpg')
            label_output_path = os.path.join(OUTPUT_PATH, 'labels', filename + '-' + str(count) + '.txt')
            visualization_output_path = os.path.join(OUTPUT_PATH, 'visualization', filename + '-' + str(count) + '.jpg')
            for label in label_list: 
                if label['startX'] > x and label['endX'] < x + 500 \
                and label['startY'] > y and label['endY'] < y + 500:
                    exist_flag = True
                    print((count, x, y, x+500, y+500))
                    print(label)
                    with open(label_output_path, 'a') as f:
                        f.write('{type} {startX} {startY} {endX} {endY}\n'.format(type=label['type'],
                                                                               startX=label['startX']-x,
                                                                               startY=label['startY']-y,
                                                                               endX=label['endX']-x,
                                                                               endY=label['endY']-y))
                    cv2.rectangle(canvas, (label['startX'], label['startY']), (label['endX'], label['endY']), (0, 255, 255), 3)
            count += 1
            if exist_flag:
                cv2.rectangle(canvas, (x, y), (x+500, y+500), (255, 255, 0), 3)
                cv2.imwrite(image_output_path, image[y:y+500, x:x+500, 0:3])
                # 通过读取输出文件生成可视化图便于检验
                test_image = cv2.imread(image_output_path)
                with open(label_output_path, 'r') as f:
                    test_labels = f.readlines()
                    for test_label in test_labels:
                        test_label = test_label.split(' ')
                        cv2.rectangle(test_image, 
                                      (int(test_label[1]), int(test_label[2])), 
                                      (int(test_label[3]), int(test_label[4])), 
                                      (0, 255, 255), 3)
                cv2.imwrite(visualization_output_path, test_image)
    cv2.imwrite(os.path.join(OUTPUT_PATH, 'visualization', filename + '.jpg'), canvas)
    print('包含样本%s，标记样本%s'%(len(label_list), count))                
                    

In [91]:
for i in os.walk(os.path.join(DATA_PATH, 'labels')):
    label_files = i[2]
    for label_file in label_files:
        label_file_path = os.path.join(DATA_PATH, 'labels', label_file)
        with tf.gfile.GFile(label_file_path, 'r') as lf:
            all_in_one = lf.readlines()
            for line in all_in_one:
                if line.find('plane') != -1 or line.find('helicopter') != -1:
                    split_image(label_file.split('.')[0])
                    break

P0000
(59, 500, 3750, 1000, 4250)
{'startX': 798, 'startY': 3915, 'endX': 976, 'endY': 4082, 'type': 'plane'}
(60, 500, 4000, 1000, 4500)
{'startX': 683, 'startY': 4182, 'endX': 864, 'endY': 4345, 'type': 'plane'}
(60, 500, 4000, 1000, 4500)
{'startX': 631, 'startY': 4322, 'endX': 813, 'endY': 4485, 'type': 'plane'}
(61, 500, 4250, 1000, 4750)
{'startX': 631, 'startY': 4322, 'endX': 813, 'endY': 4485, 'type': 'plane'}
(80, 750, 3500, 1250, 4000)
{'startX': 925, 'startY': 3742, 'endX': 1106, 'endY': 3929, 'type': 'plane'}
(81, 750, 3750, 1250, 4250)
{'startX': 798, 'startY': 3915, 'endX': 976, 'endY': 4082, 'type': 'plane'}
(125, 1250, 3750, 1750, 4250)
{'startX': 1509, 'startY': 3987, 'endX': 1640, 'endY': 4131, 'type': 'plane'}
(127, 1250, 4250, 1750, 4750)
{'startX': 1354, 'startY': 4495, 'endX': 1508, 'endY': 4649, 'type': 'plane'}
(147, 1500, 3750, 2000, 4250)
{'startX': 1509, 'startY': 3987, 'endX': 1640, 'endY': 4131, 'type': 'plane'}
(158, 1750, 1000, 2250, 1500)
{'startX': 1932

(176, 1913, 4000, 2413, 4500)
{'startX': 1917, 'startY': 4065, 'endX': 2171, 'endY': 4352, 'type': 'plane'}
包含样本31，标记样本180
P0032
(5, 0, 1189, 500, 1689)
{'startX': 7, 'startY': 1450, 'endX': 203, 'endY': 1648, 'type': 'plane'}
(5, 0, 1189, 500, 1689)
{'startX': 288, 'startY': 1432, 'endX': 468, 'endY': 1616, 'type': 'plane'}
(10, 250, 1000, 750, 1500)
{'startX': 550, 'startY': 1263, 'endX': 742, 'endY': 1457, 'type': 'plane'}
(11, 250, 1189, 750, 1689)
{'startX': 288, 'startY': 1432, 'endX': 468, 'endY': 1616, 'type': 'plane'}
(11, 250, 1189, 750, 1689)
{'startX': 550, 'startY': 1263, 'endX': 742, 'endY': 1457, 'type': 'plane'}
(16, 500, 1000, 1000, 1500)
{'startX': 692, 'startY': 1181, 'endX': 868, 'endY': 1364, 'type': 'plane'}
(16, 500, 1000, 1000, 1500)
{'startX': 550, 'startY': 1263, 'endX': 742, 'endY': 1457, 'type': 'plane'}
(17, 500, 1189, 1000, 1689)
{'startX': 550, 'startY': 1263, 'endX': 742, 'endY': 1457, 'type': 'plane'}
(21, 750, 750, 1250, 1250)
{'startX': 1054, 'startY'

包含样本11，标记样本170
P0041
(11, 250, 500, 750, 1000)
{'startX': 394, 'startY': 714, 'endX': 501, 'endY': 803, 'type': 'plane'}
(38, 1000, 500, 1500, 1000)
{'startX': 1223, 'startY': 660, 'endX': 1315, 'endY': 768, 'type': 'plane'}
(38, 1000, 500, 1500, 1000)
{'startX': 1281, 'startY': 695, 'endX': 1384, 'endY': 790, 'type': 'plane'}
(41, 1000, 1250, 1500, 1750)
{'startX': 1364, 'startY': 1453, 'endX': 1470, 'endY': 1566, 'type': 'plane'}
(47, 1250, 500, 1750, 1000)
{'startX': 1281, 'startY': 695, 'endX': 1384, 'endY': 790, 'type': 'plane'}
(49, 1250, 1000, 1750, 1500)
{'startX': 1421, 'startY': 1354, 'endX': 1522, 'endY': 1467, 'type': 'plane'}
(49, 1250, 1000, 1750, 1500)
{'startX': 1506, 'startY': 1160, 'endX': 1594, 'endY': 1264, 'type': 'plane'}
(50, 1250, 1250, 1750, 1750)
{'startX': 1573, 'startY': 1545, 'endX': 1666, 'endY': 1652, 'type': 'plane'}
(50, 1250, 1250, 1750, 1750)
{'startX': 1364, 'startY': 1453, 'endX': 1470, 'endY': 1566, 'type': 'plane'}
(50, 1250, 1250, 1750, 1750)
{'s

(24, 750, 0, 1250, 500)
{'startX': 1111, 'startY': 276, 'endX': 1197, 'endY': 369, 'type': 'plane'}
(24, 750, 0, 1250, 500)
{'startX': 1035, 'startY': 365, 'endX': 1111, 'endY': 451, 'type': 'plane'}
(25, 750, 250, 1250, 750)
{'startX': 1160, 'startY': 561, 'endX': 1241, 'endY': 644, 'type': 'plane'}
(25, 750, 250, 1250, 750)
{'startX': 1111, 'startY': 276, 'endX': 1197, 'endY': 369, 'type': 'plane'}
(25, 750, 250, 1250, 750)
{'startX': 1035, 'startY': 365, 'endX': 1111, 'endY': 451, 'type': 'plane'}
(25, 750, 250, 1250, 750)
{'startX': 1070, 'startY': 436, 'endX': 1149, 'endY': 517, 'type': 'plane'}
(25, 750, 250, 1250, 750)
{'startX': 1112, 'startY': 498, 'endX': 1189, 'endY': 584, 'type': 'plane'}
(26, 750, 500, 1250, 1000)
{'startX': 1102, 'startY': 816, 'endX': 1200, 'endY': 923, 'type': 'plane'}
(26, 750, 500, 1250, 1000)
{'startX': 1041, 'startY': 743, 'endX': 1136, 'endY': 839, 'type': 'plane'}
(26, 750, 500, 1250, 1000)
{'startX': 1160, 'startY': 561, 'endX': 1241, 'endY': 644

(51, 750, 750, 1250, 1250)
{'startX': 1135, 'startY': 1138, 'endX': 1226, 'endY': 1221, 'type': 'plane'}
(52, 750, 1000, 1250, 1500)
{'startX': 920, 'startY': 1144, 'endX': 999, 'endY': 1232, 'type': 'plane'}
(52, 750, 1000, 1250, 1500)
{'startX': 1135, 'startY': 1138, 'endX': 1226, 'endY': 1221, 'type': 'plane'}
(52, 750, 1000, 1250, 1500)
{'startX': 1123, 'startY': 1226, 'endX': 1218, 'endY': 1311, 'type': 'plane'}
(52, 750, 1000, 1250, 1500)
{'startX': 905, 'startY': 1344, 'endX': 1001, 'endY': 1433, 'type': 'plane'}
(52, 750, 1000, 1250, 1500)
{'startX': 1111, 'startY': 1385, 'endX': 1207, 'endY': 1475, 'type': 'plane'}
(53, 750, 1250, 1250, 1750)
{'startX': 905, 'startY': 1344, 'endX': 1001, 'endY': 1433, 'type': 'plane'}
(53, 750, 1250, 1250, 1750)
{'startX': 1111, 'startY': 1385, 'endX': 1207, 'endY': 1475, 'type': 'plane'}
(53, 750, 1250, 1250, 1750)
{'startX': 1106, 'startY': 1477, 'endX': 1200, 'endY': 1548, 'type': 'plane'}
(53, 750, 1250, 1250, 1750)
{'startX': 1098, 'start

...

(67, 1750, 1000, 2250, 1500)
{'startX': 1973, 'startY': 1126, 'endX': 2073, 'endY': 1219, 'type': 'plane'}
(81, 2250, 0, 2750, 500)
{'startX': 2555, 'startY': 296, 'endX': 2727, 'endY': 481, 'type': 'plane'}
(82, 2250, 250, 2750, 750)
{'startX': 2555, 'startY': 296, 'endX': 2727, 'endY': 481, 'type': 'plane'}
(90, 2486, 0, 2986, 500)
{'startX': 2555, 'startY': 296, 'endX': 2727, 'endY': 481, 'type': 'plane'}
(91, 2486, 250, 2986, 750)
{'startX': 2555, 'startY': 296, 'endX': 2727, 'endY': 481, 'type': 'plane'}
包含样本11，标记样本99
P2748
(0, 0, 0, 500, 500)
{'startX': 96, 'startY': 213, 'endX': 256, 'endY': 369, 'type': 'plane'}
(0, 0, 0, 500, 500)
{'startX': 188, 'startY': 298, 'endX': 347, 'endY': 457, 'type': 'plane'}
(0, 0, 0, 500, 500)
{'startX': 88, 'startY': 136, 'endX': 194, 'endY': 236, 'type': 'plane'}
(0, 0, 0, 500, 500)
{'startX': 19, 'startY': 191, 'endX': 125, 'endY': 295, 'type': 'plane'}
(0, 0, 0, 500, 500)
{'startX': 186, 'startY': 141, 'endX': 285, 'endY': 241, 'type': 'plane'

(25, 250, 2500, 750, 3000)
{'startX': 622, 'startY': 2547, 'endX': 745, 'endY': 2648, 'type': 'plane'}
(31, 500, 250, 1000, 750)
{'startX': 664, 'startY': 615, 'endX': 765, 'endY': 734, 'type': 'plane'}
(32, 500, 500, 1000, 1000)
{'startX': 664, 'startY': 615, 'endX': 765, 'endY': 734, 'type': 'plane'}
(33, 500, 750, 1000, 1250)
{'startX': 607, 'startY': 896, 'endX': 746, 'endY': 1033, 'type': 'plane'}
(35, 500, 1250, 1000, 1750)
{'startX': 619, 'startY': 1447, 'endX': 753, 'endY': 1576, 'type': 'plane'}
(36, 500, 1500, 1000, 2000)
{'startX': 615, 'startY': 1786, 'endX': 754, 'endY': 1914, 'type': 'plane'}
(37, 500, 1750, 1000, 2250)
{'startX': 615, 'startY': 1786, 'endX': 754, 'endY': 1914, 'type': 'plane'}
(37, 500, 1750, 1000, 2250)
{'startX': 617, 'startY': 1948, 'endX': 747, 'endY': 2079, 'type': 'plane'}
(37, 500, 1750, 1000, 2250)
{'startX': 631, 'startY': 2131, 'endX': 748, 'endY': 2229, 'type': 'plane'}
(38, 500, 2000, 1000, 2500)
{'startX': 631, 'startY': 2131, 'endX': 748, '

(16, 250, 0, 750, 500)
{'startX': 446, 'startY': 281, 'endX': 507, 'endY': 339, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 408, 'startY': 315, 'endX': 471, 'endY': 377, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 375, 'startY': 353, 'endX': 439, 'endY': 409, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 305, 'startY': 421, 'endX': 369, 'endY': 482, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 265, 'startY': 453, 'endX': 339, 'endY': 522, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 546, 'startY': 401, 'endX': 609, 'endY': 461, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 506, 'startY': 429, 'endX': 570, 'endY': 491, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 462, 'startY': 463, 'endX': 526, 'endY': 525, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 369, 'startY': 527, 'endX': 441, 'endY': 596, 'type': 'plane'}
(17, 250, 250, 750, 750)
{'startX': 604, 'startY': 511, 'endX': 665, 'endY': 568, 'type': 'plane'}
(17, 250, 25

包含样本32，标记样本460
P2804
(9, 250, 250, 750, 750)
{'startX': 357, 'startY': 274, 'endX': 617, 'endY': 504, 'type': 'plane'}
(18, 500, 500, 1000, 1000)
{'startX': 593, 'startY': 799, 'endX': 804, 'endY': 996, 'type': 'plane'}
(19, 500, 750, 1000, 1250)
{'startX': 593, 'startY': 799, 'endX': 804, 'endY': 996, 'type': 'plane'}
(28, 750, 1000, 1250, 1500)
{'startX': 801, 'startY': 1220, 'endX': 1053, 'endY': 1498, 'type': 'plane'}
(61, 1750, 1250, 2250, 1750)
{'startX': 1998, 'startY': 1493, 'endX': 2126, 'endY': 1629, 'type': 'plane'}
(67, 2000, 750, 2500, 1250)
{'startX': 2191, 'startY': 1099, 'endX': 2325, 'endY': 1222, 'type': 'plane'}
(68, 2000, 1000, 2500, 1500)
{'startX': 2117, 'startY': 1155, 'endX': 2351, 'endY': 1386, 'type': 'plane'}
(68, 2000, 1000, 2500, 1500)
{'startX': 2191, 'startY': 1099, 'endX': 2325, 'endY': 1222, 'type': 'plane'}
(68, 2000, 1000, 2500, 1500)
{'startX': 2296, 'startY': 1051, 'endX': 2496, 'endY': 1253, 'type': 'plane'}
(69, 2000, 1250, 2500, 1750)
{'startX': 

(118, 2250, 2500, 2750, 3000)
{'startX': 2380, 'startY': 2700, 'endX': 2525, 'endY': 2810, 'type': 'plane'}
(129, 2332, 2250, 2832, 2750)
{'startX': 2427, 'startY': 2653, 'endX': 2506, 'endY': 2715, 'type': 'plane'}
(129, 2332, 2250, 2832, 2750)
{'startX': 2432, 'startY': 2537, 'endX': 2505, 'endY': 2622, 'type': 'plane'}
(130, 2332, 2500, 2832, 3000)
{'startX': 2427, 'startY': 2653, 'endX': 2506, 'endY': 2715, 'type': 'plane'}
(130, 2332, 2500, 2832, 3000)
{'startX': 2432, 'startY': 2537, 'endX': 2505, 'endY': 2622, 'type': 'plane'}
(130, 2332, 2500, 2832, 3000)
{'startX': 2380, 'startY': 2700, 'endX': 2525, 'endY': 2810, 'type': 'plane'}
包含样本17，标记样本132


In [22]:
[ x for x in range(0, 601, 250)]

[0, 250, 500]